<a href="https://colab.research.google.com/github/HMS-Cleveland/Questionary_logist/blob/my/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, recall_score

!pip install shap
import shap

################修改！！！2
#指定要读取的列号（从0开始数）
usecols = [0,1,2,3,5,6,7,8,11,12]
#给这些列命名
names = ['menopause', 'age_5_years', 'BIRADS_breast_density','ethnic','bmi',
         'first_birth','num_of_first_degree_cancer','breast_surgery',
         'hormone_therapy','diagnosis']


#读取
data = pd.read_csv('/content/drive/MyDrive/公开数据集/risk.txt',
           sep=r'\s+',
           header=None,
           usecols=usecols,
           names=names)

# 处理未知值（编码9表示未知）
data.replace(9, np.nan, inplace=True)                     # 将9替换为NaN
# 计算每行缺失值个数
missing_per_row = data.isnull().sum(axis=1)
# 保留缺失值 ≤ 3 的行
data = data[missing_per_row <= 0].copy()



#按 is_train 拆分
X = data.drop(columns=['diagnosis'])
y = data['diagnosis']

# 4. 创建 8:1:1 的拆分
# 第一步：分出测试集 (10%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.1,
    stratify=y,
    random_state=42
)

# 第二步：从剩余数据中分出验证集 (10% of total, 即剩余数据的1/9)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.1111,  # 1/9 ≈ 0.1111
    stratify=y_temp,
    random_state=42
)

# 统计类别数量
count_0 = np.count_nonzero(y_train == 0)  # 统计0的数量
count_1 = np.count_nonzero(y_train == 1)  # 统计1的数量

print(f"y_train中0的数量: {count_0}")
print(f"y_train中1的数量: {count_1}")

# 统计类别数量
count_0 = np.count_nonzero(y_test == 0)  # 统计0的数量
count_1 = np.count_nonzero(y_test == 1)  # 统计1的数量

print(f"y_test中0的数量: {count_0}")
print(f"y_test中1的数量: {count_1}")

# 统计类别数量
count_0 = np.count_nonzero(y_val == 0)  # 统计0的数量
count_1 = np.count_nonzero(y_val == 1)  # 统计1的数量

print(f"y_val中0的数量: {count_0}")
print(f"y_val中1的数量: {count_1}")



neg_weight = 1 / np.sqrt(len(y_train[y_train == 0]))  # 阴性样本权重
pos_weight = 7.5 / np.sqrt(len(y_train[y_train == 1]))  # 阳性样本权重（5倍惩罚漏诊）

# 应用预处理
X_train_processed = X_train.values  # 训练集拟合转换
X_test_processed = X_test.values         # 测试集仅转换

# 7. 训练 Logistic 回归
model = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='liblinear',
    max_iter=5000,
    #class_weight='balanced',
    class_weight={0: neg_weight, 1: pos_weight},  # 动态权重
)
model.fit(X_train_processed, y_train)

# 8. 验证集评估（调参用）
val_prob = model.predict_proba(X_val)[:, 1]

val_auc = roc_auc_score(y_val, val_prob)
print(f'验证集 AUC = {val_auc:.3f}')

print(y_train.value_counts(normalize=True))

#使用 LinearExplainer（专为线性模型优化）
explainer = shap.LinearExplainer(
    model,
    X_train_processed,  # 训练集数据
    feature_names=X_train.columns.tolist()  # 特征名称
)
###########################################################
# 计算测试集样本的 SHAP 值
shap_values = explainer.shap_values(X_test_processed)

# === 全局特征贡献（所有样本） ===
# 1. 特征重要性条形图（按平均|SHAP|排序）
shap.summary_plot(shap_values, X_test_processed, feature_names=X_train.columns, plot_type="bar")

# 2. 蜂群图（展示特征值与贡献的关系）
shap.summary_plot(shap_values, X_test_processed, feature_names=X_train.columns)

# === 局部特征贡献（单个样本） ===
# 3. 瀑布图（解释单个预测）
sample_idx = 0  # 选择一个样本
shap.force_plot(
    explainer.expected_value,
    shap_values[sample_idx, :],
    X_test_processed[sample_idx, :],
    feature_names=X_train.columns,
    matplotlib=True  # 在 Colab 中显示
)
############################################################
# 9. 测试集最终评估（锁定模型后）
test_prob = model.predict_proba(X_test)[:, 1]
val_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, test_prob)
print(f'测试集 AUC = {test_auc:.3f}')
print(classification_report(y_test, (test_prob > 0.5).astype(int)))


y_true = y_test
accuracy = accuracy_score(y_test,val_pred)
print("============")
print(f"准确率 = {accuracy:.3f}")

# 用你已有的测试集标签和预测结果
report = classification_report(
    y_test,
    val_pred,           # 你之前已经用 0.5 阈值得到的硬标签
    output_dict=True    # 返回字典
)

# 打印看看
print(report)

# 灵敏度（召回率）= 真阳性率
sensitivity = report['1']['recall']
print(f"Sensitivity (Recall for class 1) = {sensitivity:.3f}")

# 特异性 = 真阴性率 = Recall for class 0
specificity = report['0']['recall']
print(f"Specificity (Recall for class 0) = {specificity:.3f}")

# 10. 相对风险(OR) 与 绝对风险
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'log_OR': model.coef_[0],
    'OR': np.exp(model.coef_[0])
}).sort_values('OR', ascending=False)


# 11. 给某个人计算 5 年绝对风险示例
# person 为 1 行 0/1 特征向量
person = np.array([[1, 0, 1, 0, 0, 1, 0, 1, 0]])  # 示例
logit = model.decision_function(person)[0]
abs_risk = 1 / (1 + np.exp(-logit))
print(f'该个体绝对风险 = {abs_risk:.3%}')


# 特征重要性解读（医疗决策关键！）
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Odds_Ratio': np.exp(model.coef_[0])
}).sort_values('Odds_Ratio', ascending=False).reset_index(drop=True)

print("\n【风险因素OR值排名】")
print(coef_df.head(10))